# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [66]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice
import pandas as pd
import re

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [67]:

def dna_to_rna(s):
    rules = {"A":"A","C":"C","G":"G","T":"U"}
    res = []
    for sym in s:
        res.append(rules[sym])
    return "".join(res)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))
    print(dna_to_rna(""))
    print(dna_to_rna("T"))

AACGUGAUUUC

U


### Idea of solution

Since we were asked to use a dictionary to store the rules, I decided to use a for loop that maps each character using the dictionary as a mapping.

### Discussion

The function is correctly converting a string of any given length that represents a DNA sequence to an RNA sequence.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [68]:

def get_dict():
    with open("CodonUsageTable.html","r") as f:
        content = f.read()
    table = re.findall(r"[AUGC][AUGC][AUGC] [A-Z|\*]",content)
    result = {}
    for row in table:
        pair = row.split()
        codon = pair[0]
        aa = pair[1]
        result[codon] = aa
    return result

    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)
    print("AAA converts to", codon_to_aa["AAA"])
    print("UAG converts to", codon_to_aa["UAG"])
    print("UGG converts to", codon_to_aa["UGG"])
    print("The total number of codons is", len(codon_to_aa))

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}
AAA converts to K
UAG converts to *
UGG converts to W
The total number of codons is 64


### Idea of solution
Since the data table was the only text in the webpage that was formatted in that way, I used a regular expression to compile all phrases in the page that consisted of a codon followed by an amino acid. This gave me a tuple of strings where each string contained the codon and the amino acid seperated by a space. I then used a loop to add each string into the dictionary with the codn as the key and the amino acid as the value and returned the dictionary.

### Discussion
The dictionary has 64 keys which means the regular expression found all the codons as required. ALso, upon trying random codons, we see that they are being translated to the correct amino acid. 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [69]:

def get_dict_list():
    with open("CodonUsageTable.html","r") as f:
        content = f.read()
    table = re.findall(r"[AUGC][AUGC][AUGC] [A-Z|\*]",content)
    result = {}
    for row in table:
        pair = row.split()
        codon = pair[0]
        aa = pair[1]
        if aa in result:
            result[aa].append(codon)
        else:
            result[aa] = [codon]
        
    return result
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)
    print("C comes from the following codons: ", aa_to_codons["C"])
    print("* comes from the following codons: ", aa_to_codons["*"])
    total_codons = 0
    for aa in aa_to_codons:
        total_codons += len(aa_to_codons[aa])
    print("The total number of codons is", total_codons)
    


{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}
C comes from the following codons:  ['UGU', 'UGC']
* comes from the following codons:  ['UAA', 'UGA', 'UAG']
The total number of codons is 64


### Idea of solution

Just like in question 2, I used a regular expression to get all the pairs of codons and amino acids as strings containing both of them seperated by a space. I then used a loop to add each unique amino acid to my resulting dictionary as a key if it does not already exist in the dictionary and add the codon that produces it to the list which is its value. So in each step of the loop, one codon gets added to the list of one amino acid from the dictionary or a new key-value pair gets created with the codon-amino-acid pair.

### Discussion

I counted the total number of codons by adding the length of each value in the dictionary that results from the get_dict_list function. The total comes to 64 which tells me that all codons are included. This is not necessary since the codons may have erroniously repeated so to be certain, I should have merged all the lists together into one list, converted it to a set and then measured the length. However, from lookingat the dictionary and checking the values of some arbitrary amino acids, the functions appear to have the correct output.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [70]:

def rna_to_prot(s):
    result = ""
    for i in range(0,len(s),3):
        result += get_dict()[s[i:i+3]]
    return result

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print("ATGATATCATCGACGATGTAG translates to ",dna_to_prot("ATGATATCATCGACGATGTAG"))
    print("ACG translates to ", dna_to_prot("ACG"))
    print("AUGCGCUUUGCAGCC translates to ", rna_to_prot("AUGCGCUUUGCAGCC"))
    

ATGATATCATCGACGATGTAG translates to  MISSTM*
ACG translates to  T
AUGCGCUUUGCAGCC translates to  MRFAA


### Idea of solution

The function rna_to_prot runs a for loop through the string s that is made up of codons (which are three characters each). The loop looks up each codon of s in the dictionary resulting from get_dict() which maps each codon to its corresponding amino acid. The value of the key is then appended to the resulting string which is the output of the function.

### Discussion

The function runs correctly for a sequence of one or more codons of dna and rna sequences.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [71]:

def get_probabability_dict():
    with open("CodonUsageTable.html","r") as f:
        content = f.read()
    table = re.findall(r'([AUGC][AUGC][AUGC])\s+([A-Z|\*])\s+\d+\.\d+\s+\d+\.\d+\s+\(\s*(\d+)\s*\)',content)
    aa_totals = {}
    for row in table:
        if row[1] in aa_totals:
            aa_totals[row[1]] += int(row[2])
        else:
            aa_totals[row[1]] = int(row[2])
    prob_dict = {}
    for row in table:
        prob_dict[row[0]] = int(row[2])/aa_totals[row[1]]
    return prob_dict
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))
    print("The number of codons are ", len(get_probabability_dict()))

    #printing the fraction column instead to see if the calculations match:
    print("\nHere are the truncated fraction values from the Codon Usage Table:")
    with open("CodonUsageTable.html","r") as f:
        content = f.read()
    fractions = re.findall(r'([AUGC][AUGC][AUGC])\s+[A-Z|\*]\s+(\d+\.\d+)',content)
    usage_table = {}
    for row in fractions:
        usage_table[row[0]] = float(row[1])
    usage_table_items = sorted(usage_table.items(), key=lambda x: x[0])
    for i in range(1 + len(usage_table_items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in usage_table_items[i*6:6+i*6]
        ))





AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134
The number of codons are  64

Here are the truncated fraction values from the Codon Usage Table:
AAA: 0.

### Idea of solution

I first used a regular expression to find all the triples in the html file correspinding to the name of the codon, the amino acid it encodes, and its frequency. Next, by running a for loop through this list of triples, I created a dictionary called aa_totals where the keys are all the possible amino acids and their values are the sum of frequencies of all codons encoding them. Finally, to create the probability dictionary, I ran a for loop through the list of triples. I set the keys to be the codons and their values to be thier frequency divided by the value of the amino acid it encodes from the aa_totals dictionary.

### Discussion

To test it out, I simply printed the resulting dictionary and also checked that its length is 64 as that is the number of possibilities of codons. I also used a regular expression to print the fraction column from the html page as a dictionary, which represents the same values as the values in get_probability_dict but rounded to two decimal places. By looking at both the dictionaries side by side we see that the values in both dictionaries are equal  when rounded to 2 decimal places so the calculations are correct.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [72]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        codon_probs = get_probabability_dict()
        aa_to_codons = get_dict_list()
        rna = ""
        for aa in s:
            max_codon = aa_to_codons[aa][0]
            for codon in aa_to_codons[aa]:
                if codon_probs[max_codon] < codon_probs[codon]:
                    max_codon = codon
            rna += max_codon
        return rna


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print("LTPIQNRA converts to", protein_to_rna.convert("LTPIQNRA"))
    protein_to_rna2 = ProteinToMaxRNA()
    print("Empty string converts to", protein_to_rna2.convert(""))
    protein_to_rna3 = ProteinToMaxRNA()
    star_to_rna = protein_to_rna2.convert("*")
    print("* converts to", protein_to_rna2.convert("*"))
    print("In aa_to_codons dictionary, * has this value:", get_dict_list()["*"])
    print("The probabilities of the codons encoded by * are: ", list(get_probabability_dict()[i] for i in get_dict_list()["*"]))




LTPIQNRA converts to CUGACCCCCAUCCAGAACAGAGCC
Empty string converts to 
* converts to UGA
In aa_to_codons dictionary, * has this value: ['UAA', 'UGA', 'UAG']
The probabilities of the codons encoded by * are:  [0.29701911804823383, 0.46624296805302623, 0.23673791389873997]


### Idea of solution

For each amino acid in s which represents a protein, I looked up the amino acid in the dictionary get_dict_list() to get the list of all codons that encode it. I then looked up each of these codons in the dictionary get_probability_dict() to compare their probabilities and find the one with the highest probability. The codon with the highest probability was appended to my resulting string rna in which I accumulated the codons with highest probability for each amino acid.

I did not edit the init function as I could not think of any attributes I might need to initialise in order to complete the code for the convert method. 

### Discussion

The given example "LTPIQNRA" outputs a valid rna sequence however we do not know from this example if the output is the most probable one. So we run another test on a single amino acid, "*". Upon looking up the probabilities of each of the codons that encode * using the dictionaries get_dict_list() and get_probability_dict(), we see that the correct codon is being returned by our method.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [73]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    cumulative = {}
    so_far = 0
    for event in dist:
        cumulative[event] = (so_far, dist[event] + so_far)
        so_far += dist[event]
    random_gen = np.random.uniform()
    for event in dist:
        if cumulative[event][0]<= random_gen < cumulative[event][1]:
            return event

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    test_output = ", ".join(random_event(distribution) for _ in range(29))
    print(test_output)
    print("Distribution: ", "\nA: ", test_output.count("A")/29, "\nC: ", test_output.count("C")/29, "\nG: ", test_output.count("G")/29, 
    "\nT: ", test_output.count("T")/29)

    distribution2 = dict(zip("ACGT", [0, 0, 0, 1]))
    test_output2 = ", ".join(random_event(distribution2) for _ in range(29))
    print(test_output2)
    print("Distribution: ", "\nA: ", test_output2.count("A")/29, "\nC: ", test_output2.count("C")/29, "\nG: ", test_output2.count("G")/29, 
    "\nT: ", test_output2.count("T")/29)

G, T, C, A, G, G, C, C, G, C, T, C, G, G, C, T, T, T, C, A, C, T, C, C, T, G, T, C, A
Distribution:  
A:  0.10344827586206896 
C:  0.3793103448275862 
G:  0.2413793103448276 
T:  0.27586206896551724
T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T, T
Distribution:  
A:  0.0 
C:  0.0 
G:  0.0 
T:  1.0


### Idea of solution

I first split up the interval [0,1) into intervals of sizes given by the distribution in dist. To do so, I created a new dictionary with the same keys as dist but for each key, I added the probabilites of the previous keys to the probability of each key. These intervals are represented as values in my new dictionary, "cumulative", in the form of lists containing two values, the lower bound and the upper bound.

Afer this, I generated a random number in the interval [0,1) using the given function and looked at each value in my cumulative dictionary to find the key whose interval contains the generated number (upper bound is excluded). This key is returned. 

### Discussion

In order to test this function, I generated bases with two distributions and for each one, I calculated the proportion of each base to see if it is roughly close to the input distributions. I ran the function a few times since the random generation is always difference and for the first test, I always get something close to the input distribution. The second distribution is trivial is so the proportion is always the same as the input distribution.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [74]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        result = ""
        for aa in s:
            codons = get_dict_list()[aa]
            dist = {}
            for codon in codons:
                dist[codon] = get_probabability_dict()[codon]
            result += random_event(dist)
        return result
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))
    print("The length of the input is ", len("LTPIQNRA"))
    print("the number of codons in the output is", len(protein_to_random_codons.convert("LTPIQRNA"))/3)
    print(protein_to_random_codons.convert("L"))
    print(get_dict_list()["L"])
    print(protein_to_random_codons.convert("I"))
    print(get_dict_list()["I"])
    print(protein_to_random_codons.convert("N"))
    print(get_dict_list()["N"])

CUUACUCCCAUACAAAACAGGGCC
The length of the input is  8
the number of codons in the output is 8.0
CUC
['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG']
AUC
['AUU', 'AUC', 'AUA']
AAU
['AAU', 'AAC']


### Idea of solution

For each amino acid in s, I looked it up in the dictionary get_dict_list() to get the list of codons that encode it. Then, I created a distribution dictionary for each of the codons by looking up each codon in the dictionary get_probabilty_dict(). The distribution dictionary contained the codons as keys and their probabilities as values. I ran the function random_event with this distribution as its input to get a random codon based on the distribution. I appended each resulting codon into the resulting string.

### Discussion

The first test is the given protein sequence which looks like a valid rna sequence since it contains codons and the correct number of codons. To check if the codons are those that encode the amino acid, we run this function on three arbitrary amino acids and check whether the output is one of the values in the list associated with that amino acid in get_dict_list(). The three amino acids passed the test.

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [75]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    first_window = s[:k]
    frequencies = {}
    frequencies["A"] = first_window.count("A")
    frequencies["G"] = first_window.count("G")
    frequencies["C"] = first_window.count("C")
    frequencies["T"] = first_window.count("T")
    
    for i in range(len(s)-k+1):
        yield frequencies
        frequencies[s[i]] -= 1
        if i+k < len(s):
            frequencies[s[i+k]] += 1

if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        count = 0
        for base in d:
            count += d[base]
        print(d, "total: ", count)
    print("The no. of dictionaries is ", len(list(sliding_window(s,4))))
    print("len(s): ", len(s))

    print("----------")

    t = "ACG"
    for d in sliding_window(t, 3):
        print(d, end=" ")
        count = 0
        for base in d:
            count += d[base]
        print("total: ", count)

{'A': 0, 'G': 0, 'C': 3, 'T': 1} total:  4
{'A': 0, 'G': 1, 'C': 3, 'T': 0} total:  4
{'A': 1, 'G': 1, 'C': 2, 'T': 0} total:  4
{'A': 1, 'G': 1, 'C': 2, 'T': 0} total:  4
{'A': 1, 'G': 2, 'C': 1, 'T': 0} total:  4
{'A': 1, 'G': 2, 'C': 1, 'T': 0} total:  4
{'A': 0, 'G': 2, 'C': 2, 'T': 0} total:  4
{'A': 0, 'G': 2, 'C': 2, 'T': 0} total:  4
{'A': 0, 'G': 1, 'C': 2, 'T': 1} total:  4
{'A': 0, 'G': 0, 'C': 2, 'T': 2} total:  4
{'A': 0, 'G': 1, 'C': 1, 'T': 2} total:  4
{'A': 0, 'G': 1, 'C': 1, 'T': 2} total:  4
{'A': 0, 'G': 1, 'C': 2, 'T': 1} total:  4
The no. of dictionaries is  13
len(s):  16
----------
{'A': 1, 'G': 1, 'C': 1, 'T': 0} total:  3


### Idea of solution

As suggested above, I start by counting the occurences of each base in the first k bases of the string s which is the substring of s with starting position 0 and ending position k-1. I made this into the dictionary for the first window. For each remaining window, I looped through the possible starting positions which go from 0 to len(s)-k, and modified the dictionary based on the position. For the first iteration, the dictionary yielded is the one created above for the first window. Before yielding the dictionary for the next window, we reduce the count of the first character froom the current window and increase the count of the last character of the next window in the dictionary that was yielded. After making these changes, the new dictionary is yielded. The same continues until we reach the last window.

### Discussion

The given example outputs the right total count in each dictionary. We also see that it produces the right number of dictionaries since there are 16 characters so the last window of length 4 would start at the 13th character.

Finally, to check if the counts are right, we try a string s with only one window and can observe that the counts for the dictionary outputted are correct.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [76]:
def context_list(s, k):
    CL = {}
    for i in range(len(s)-k):
        curr_kmer = s[i:i+k]
        if curr_kmer in CL:
            CL[curr_kmer] += s[i+k]
        else:
            CL[curr_kmer] = s[i+k]
    return CL
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    total = 0
    for kmer in d:
        total += len(d[kmer])
    print(d)
    print("total: ", total)
    print("len(s): ", len(s))
    print(context_list(s,0))

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}
total:  19
len(s):  21
{'': 'ATGATATCATCGACGATCTAG'}


### Idea of solution

I created an empty dictioanry and then looped through each substring of s of size k. If the substring was not int he dictionary already, I added it as a key and its value was a string containing the character following the substring. If the substring was already in the dictionary, I updated its value by appending the character following the substring to the string which was its current value. This dictionary is returned.

### Discussion

The function successfully outputs a dictionary with all 2-mers in the given string as keys. The total number of symbols accumulated by all the 2-mers adds up to 19 which is correct since the first and second characters do not follow any 2-mers and there are 21 characters in total. We also see that setting k to 0 gives one possible k-mer which is the empty string and the value is a concatenation of every character in s which is also correct.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [77]:
def context_probabilities(s, k):
    CL = context_list(s,k)
    CP = {}
    for context in CL:
        symbols = CL[context]
        total = len(symbols)
        prob_A = symbols.count("A")/total
        prob_C = symbols.count("C")/total
        prob_G = symbols.count("G")/total
        prob_T = symbols.count("T")/total
        CP[context] = {"A":prob_A, "C":prob_C, "G":prob_G, "T":prob_T}
    return CP
    
if __name__ == '__main__':
    k1 = 0
    k2 = 2
    s = "ATGATATCATCGACGATGTAG"
    print(context_probabilities(s,k2))
    print(context_list(s,k2))
    print(context_probabilities(s,k1))
    print("A:", s.count("A"), "C:", s.count("C"), "G:", s.count("G"), "T:", s.count("T"))
    print("total:", len(s))

{'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4, 'T': 0.0}, 'TG': {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.5}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666}, 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'G': 0.5, 'T': 0.0}, 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}, 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}
{'AT': 'GACCG', 'TG': 'AT', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AG', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'GT': 'A'}
{'': {'A': 0.3333333333333333, 'C': 0.14285714285714285, 'G': 0.23809523809523808, 'T': 0.2857142857142857}}
A: 7 C: 3 G: 5 T: 6
total: 21


### Idea of solution

I first ran the function context_list(s,k) to get the dictionary of all k-mers in s whose values are the concatenation of all symbols following the k-mer in s. I ran a for loop such that each k-mer, we get its value from context_list and use the string method count to count the occurences of each of the 4 bases. I divided these counts by the length of the concatenation which is the total number of symbols that followed these k-mers. I then created a dictionary containing each base as a key and the aforementioned quotient as its value. Finally, I created a dictionary and added each k-mer from the above for loop into this dictionary as a key whose value is set to the dictionary of probabilities that I computed. This dictionary is returned.

### Discussion
I ran the context_probabilities function on the given string s and also the context_list function to compare the probabilities. 'AT' has the value 'GACCG' in context_list so the distributions should be 2/5 for G and C, 1/5 for A and 0 for T and we can see that this has been correctly outputted.

When k is set to 0, the dictionary that is the key of the empty string should just give the proportion of the string that contains each base. By looking at the counts of each base in string s dividing it by the length of s, we see that values in the dictionary are correct.


12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [78]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        np.random.seed(seed)
        res = ""
        if n < self.k:
            first_few = n
        else:
            first_few = self.k
        for i in range(first_few):
            res += random_event(self.zeroth)
        while len(res) < n:
            context = res[-1*self.k:]
            if context in self.kth:
                res += random_event(self.kth[context])
            else:
                res += random_event(self.zeroth)
        return res

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
   
    mc = MarkovChain(zeroth, kth)
    print("A sequence of length 10: ", mc.generate(10, 0))

    mc = MarkovChain(zeroth, kth)
    print("A sequence of length 0: ", mc.generate(0, 0))

    mc = MarkovChain(zeroth, kth)
    print("A sequence of length less than k: ", mc.generate(1, 0)) 


A sequence of length 10:  TGTATGATGA
A sequence of length 0:  
A sequence of length less than k:  T


### Idea of solution
We are required to generate a string of n characters. For the first k characters (or for all the characters if n is less than k), I used the probability distribution given in zeroth and plugged it into the function random_event which generates one of the events in the given dictionary based on their probabilities which are the values of the dictionary. To generate the remaining characters, I used a while loop that appended a character generated by random_event with the kth dictionary as its input to the resulting string until the resulting string (res) reached a length of n. This resulting string was returned once the loop ended. 

### Discussion

We can see that the generate method successfully generates a sequence of length 10 and also one of length 1 which is less than k=2. It also works for a input length 0 and correctly generates an empty string. It would be tough to check whether the generation is coherent with the given input distribution as it is randomly generated, and I am unable to test the same.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [79]:
def context_pseudo_probabilities(s, k):
    CL = context_list(s,k)
    CP = {}
    all_kmers = []
    for i in product("ACGT", repeat=k):
        all_kmers.append("".join(i))
    for kmer in all_kmers:
        CP[kmer] = {"A":1, "C":1, "G":1, "T":1}
    for context in CL:
        symbols = CL[context]
        CP[context]['A'] += symbols.count("A")
        CP[context]['C'] += symbols.count("C")
        CP[context]['G'] += symbols.count("G")
        CP[context]['T'] += symbols.count("T")
    for context in CP:
        total = CP[context]['A']+CP[context]['C']+CP[context]['G']+CP[context]['T']
        CP[context]['A'] /= total
        CP[context]['C'] /= total
        CP[context]['G'] /= total
        CP[context]['T'] /= total
    
    return CP
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth_2 = context_pseudo_probabilities(s, k)
    zeroth2 = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth2}")
    print("Counts: A:", s.count("A")+1, "C:", s.count("C")+1, "G:", s.count("G")+1, "T:", s.count("T")+1, "total:", len(s)+4)
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth_2.items()))
    print("context list of CG:", context_list(s,k)["CG"])
    print("\n", MarkovChain(zeroth2, kth_2, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
Counts: A: 8 C: 4 G: 6 T: 7 total: 25
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333

### Idea of solution

I first got a list of all the k-mers by using the product function and looping through each tuple and converting it into a string k-mer before appending it to my list of k-mers. I then ran a for loop through my list of k-mers and created a dictionary that contains the k-mers as keys and a dictionary containing the frequency count of A,C,G and T as their values. In this for loop, I set the frequency count of every letter for every k-mer to be 1. I ran another for loop through the context list of the given string which I obtained using the context_list function. For each k-mer in the context list, its value contains a concatenation of all letters that followed this k-mer in s. I counted the frequency of each letter in this concatenation during the count method and then added the count to the respective frequency dictionary. 
Finally, I ran another for loop through the dictionary of k-mers I have and divided each frequency count in the values of the dictionary by the sum of the frequencies for each k-mer.


### Discussion

First, to check the zeroth dictionary, I printed the frequency of each letter in the string s and added 1 to account the initialization of frequencies to 1. I also printed the total length of the string s plus 4 to account for the initialization. Comparing the numbers 0.32,0.16,0.24,0.28 to the respective frequency ratios 8/25, 4/25, 6/25, 7/25, we see that the distribution is correct as these are equivalent.

Any 2-mer that did not appear in s such as AC and AG have the correct distribution of 0.25 each which they get from the initial values. I picked the 2-mer CG at random to check whether a 2-mer that is in s has the correct distribution. The context list of CG (printed above) is "AA", so the total counts of A,C,G,T in this function should be 3,1,1,1 respectively, which would give a distribution of 3/6,1/6,1/6 and 1/6. We see that the distribution for CG in the output is equivalent to the aforementioned values as desired.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [91]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        prob  = 1
        if len(s) < self.k:
            for i in range(len(s)):
                prob *= self.zeroth[s[i]]
        else:
            for i in range(self.k):
                prob *= self.zeroth[s[i]]
            for i in range(self.k,len(s)):
                prob *= self.kth[s[i-self.k:i]][s[i]]
        return prob

    
if __name__ == '__main__':
    mc = MarkovProb(2, zeroth2, kth_2)
    print(f"Probability of sequence {s} is {mc.probability(s)}")

    print(zeroth2)
    s1 = "G"
    print(f"Probability of sequence {s1} is {mc.probability(s1)}")

    s2 = "GT"
    print(f"Probability of sequence {s2} is {mc.probability(s2)} and should be", zeroth2["G"]*zeroth2["T"])

    print(kth_2["AC"])
    s3 = "ACC"
    print(f"Probability of sequence {s3} is {mc.probability(s3)} and should be", zeroth2["A"]*zeroth2["C"]*kth_2["AC"]["C"])


Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10
{'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
Probability of sequence G is 0.24
Probability of sequence GT is 0.06720000000000001 and should be 0.06720000000000001
{'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
Probability of sequence ACC is 0.01024 and should be 0.01024


### Idea of solution

If string s has length smaller than k, then all probabilities would given bu the zeroth dictionary. So i ran a for loop for each character for s and looked up the character in the zeroth dictionary. Each value looked up was multiplied together giving the probability.
Otherwise, I did the same thing as mentioned above for the first k characters of s using a for loop for them. For the remaining characters of s, I ran a for loop for each remaining character i, where I first looked up the substring formed by the k characters before i in the kth dictionary to get the probability distribution of each base and then looked up the base i in this distribution to get its probability. This number was multiplitied to the probability variable which was the result of the function.

### Discussion

Although the given example, ATGATATCATCGACGATGTAG, outputs a probability that would make sense, it would be tedious to check if the product is accurate. So I tested examples of length 1,2 and 3 so that I can manually check the probability and also because 1 is less than k and would test a different case in the code. For the string of length 1, I simply looked up the character in the zeroth dictionary to get the expected answer which indeed matches the output. I did the same for the characters in the string of length 2 and multiplied their probabilities together, which also matches the output. For the string of length three, I got the first two probabilities by looking up the first two characters in the zeroth dictionary and to get the probability of the third character, I looked up its probability in the dictionary that is the probability distribution of the first two characters in the kth dictionary. I multiplied these three values which also matched the output as desired.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [81]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        prob  = 0
        if len(s) < self.k:
            for i in range(len(s)):
                prob += np.log2(self.zeroth[s[i]])
        else:
            for i in range(self.k):
                prob += np.log2(self.zeroth[s[i]])
            for i in range(self.k,len(s)):
                prob += np.log2(self.kth[s[i-self.k:i]][s[i]])
        return prob
    
if __name__ == '__main__':
    mc = MarkovLog(2, zeroth2, kth_2)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

    print(zeroth2)
    s1 = "G"
    print(f"Log probability of sequence {s1} is {mc.log_probability(s1)} and should be {np.log2(zeroth2[s1])}")

    s2 = "GT"
    print(f"Log probability of sequence {s2} is {mc.log_probability(s2)} and should be", np.log2(zeroth2["G"])+np.log2(zeroth2["T"]))

    print(kth_2["AC"])
    s3 = "ACC"
    print(f"Log probability of sequence {s3} is {mc.log_probability(s3)} and should be", np.log2(zeroth2["A"])+np.log2(zeroth2["C"])+np.log2(kth_2["AC"]["C"]))

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315
{'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
Log probability of sequence G is -2.0588936890535687 and should be -2.0588936890535687
Log probability of sequence GT is -3.895394956770689 and should be -3.895394956770689
{'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
Log probability of sequence ACC is -6.609640474436812 and should be -6.609640474436812


### Idea of solution

I used a similar approach as the previous question in this question. For the case that the size of s is smaller than k, I found the log of the probability of each character in s where the probability was obtained by looking up the character in the zeroth dictionary. These values were then added up and returned. Otherwise, I ran a loop that does the same as in the previous case for the first k characters, and for the remaining characters, I looked up the substring formed by the k characters before i in the kth dictionary and then looked up the character i in the resulting dictionary. This would give me a probability which I then found the log of and added to the output value to be returned.

### Discussion

Just like the previous question, the first test is difficult to verify as it is a long string but we see that it does output a value that looks roughly correct. I tested the function on three strings of length 1,2,3 to check all the cases of my code and also because I can manually compute these values and compare them to the output. I looked up the characters in the zeroth and kth dictionaries as needed to get the probabilities I need, found their log and added them together. We see that the function gives the right answer for these three tests.

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [82]:
def better_context_probabilities(s, k):
    CL = {}

    for i in range(len(s)-k):
        curr_kmer = s[i:i+k]
        if curr_kmer in CL:
            CL[curr_kmer][s[i+k]] += 1
        else:
            CL[curr_kmer] = {"A":1, "C":1, "G":1, "T":1}
            CL[curr_kmer][s[i+k]] += 1
    for i in product("ACGT", repeat=k):
        kmer = "".join(i)
        if kmer not in CL:
            CL[kmer] = {"A":1, "C":1, "G":1, "T":1}

    CP = {}
    for context in CL:
        symbols = CL[context]
        total = symbols["A"]+symbols["C"]+symbols["G"]+symbols["T"]
        prob_A = symbols["A"]/total
        prob_C = symbols["C"]/total
        prob_G = symbols["G"]/total
        prob_T = symbols["T"]/total
        CP[context] = {"A":prob_A, "C":prob_C, "G":prob_G, "T":prob_T}
    return CP

if __name__ == '__main__':
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))
    print("Here are the kmers not in s:")
    for kmer in d: 
        if kmer not in s: 
            print(kmer)
    print("Let's check the bases that follow CA:", end=" ")
    print(re.findall(r"CA([ACGT])", s))

AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.3333333333333333}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T':

### Idea of solution

I first looped through each k-mer in the s to count the frequencies of each base after each k-mer. For each k-mer, I increased the count of the base proceeding it by 1 in the dictionary that is the value of the k-mer in my frequency dictionary, CL. If that k-mer is not already a key in my dictionary, I put it in and initialized the counts to 1 as suggested, before adding 1 for the base that proceeds the k-mer in s. By the end of this loop, I had a dictionary of counts of each base that proceeded each k-mer. This loop might not have added a key for every possible k-mer since s might not contain all possible k-mers. So I looped through all possible k-mers which I obtained using the product function and added any k-mers that were not already added, initializing the counts to 1.

I then looped through  each k-mer in my counts dictionary CL, added up the counts of each base to get the total and divided the total by each of the four counts to change the counts into probability. Due to the initialization being 1 for each base, the total of the counts would have never been 0 so I did not need to worry about zero division error. 

### Discussion
Note that for all the k-mers not in s, the counts would have been 1 which implies the probabilities would have been 0.25 for each base and that is indeed the case as we can see.

To check an arbitrary base other than the ones that are not in s, let us check CA. Looking at all the characters that follow CA (printed above), we see that T is the only one. So the counts for A,C,G,T would be 1,1,1 and 2 respectively, which would give probabilities of 0.2,0.2,0.2 and 0.4 which is correct.


While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [83]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.s = s
        self.k = k
        

    def generate(self, n, seed=None):
        np.random.seed(seed)
        CL = context_list(self.s,self.k)
        res = ""
        if n<self.k:
            for i in range(n):
                res += np.random.choice(np.array(list(self.s)))
        else:
            for i in range(self.k):
                res += np.random.choice(np.array(list(self.s)))
            while len(res) < n:
                context = res[-1*self.k:]
                if context in CL:
                    res += np.random.choice(np.array(list(CL[context])))
                else:
                    res += np.random.choice(np.array(list(self.s)))
        return res
        
if __name__ == '__main__':
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(10, 7))
    print(mc.generate(3, 7))
    print(mc.generate(1, 7))
    print(mc.generate(0, 7))


ATATCGATAT
ATA
A



### Idea of solution

To generate the first k characters or for all characters if n is less than k, I ran a loop where each time, I used np.random.choice to choose a random character of s by converting s into an array of its characters. This array represents the distribution of each base in s. I concatenated each generated character into a string called res. For the reamaining characters of my resulting string, I ran a while loop that concatenated a character to res until res has the correct length of n. To generate this character, I used np.random.choice again but this time, I used the string obtained by looking up the substring of tf last k characters of res so far in the context_list dictionary to get a string representing the distribution of bases. Plugging in this string into choice, I was able to generate the character. All the generated characters were concatenated to res which was the output. By using random.seed, I made sure that the random generation was always the same given a seed.

### Discussion

I was not sure how to test whether the randomly generated sequence followed the correct distributions as it would require many manual computations. We can however see that random dna sequences of the right length are being generated. Also, running it multiple times is producing the same generation which means the seed is working correctly.

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [84]:
def kmer_index(s, k):
    index_dict = {}
    for i in range(len(s)-k+1):
        curr_kmer = s[i:i+k]
        if curr_kmer in index_dict:
            index_dict[curr_kmer].append(i)
        else:
            index_dict[curr_kmer] = [i]
    return index_dict

if __name__ == '__main__':
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))
    all_indices = []
    for kmer in d: all_indices += d[kmer]
    print("all indices in dictionary: ", sorted(all_indices))
    print("length of s: ", len(s))
    

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AT': [0, 3, 5, 8, 15], 'TG': [1, 16], 'GA': [2, 11, 14], 'TA': [4, 18], 'TC': [6, 9], 'CA': [7], 'CG': [10, 13], 'AC': [12], 'GT': [17], 'AG': [19]}
all indices in dictionary:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
length of s:  21


### Idea of solution

I initialized and empty dictionary and looped through the starting position of each k-mer in s which goes from 0 to len(s)-k. If the k-mer is not in the dictionary already, I added it to the dictionary with its value being a list containing the starting position of the k-mer. If the k-mer is in the dictinary already, I appended the starting position of the current k-mer to the list that is the value of this k-mer in the dictionary. This dictionary is returned.

### Discussion

Using the numbers printed below the string s, we can check that the correct indices have been appended to the values of each k-mer. I also checked if all the indices have been accounted for in all the lists in the dictionary and indeed we see that all the possible indices (0-19) are included. The length of s is 21 so there are 19 2-mers in s.

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [85]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    codon_prob_dict = {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    for i in range(len(rna)-2):
        codon_prob_dict[rna[i:i+3]] += 1
    total_3mers = len(rna)-2
    for codon in codon_prob_dict:
        codon_prob_dict[codon] /= total_3mers
    return codon_prob_dict
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    KL_divergence = 0
    for i in p:
        if p[i] != 0.0:
            KL_divergence += p[i]*np.log2(p[i]/q[i])
    return KL_divergence

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))   
    prot_to_rna = ProteinToRandomRNA()
    assoc_rna = prot_to_rna.convert(protein)
    
    # Maybe check that converting back to protein results in the same sequence
    assoc_protein = rna_to_prot(assoc_rna)
    print("Testing if the proteins are the same: ",protein == assoc_protein)
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities(assoc_rna) # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = get_probabability_dict()# placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    random_rna = "".join(choice(list("AGCU"), n))
    cp_uniform = codon_probabilities(random_rna) # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

Testing if the proteins are the same:  True
d(original || predicted) = 95.08108417744201
d(predicted || original) = -4.243547580723664

d(original || uniform) = 97.80864294334887
d(uniform || original) = -4.102470506137899

d(predicted || uniform) = 0.06900172357165896
d(uniform || predicted) = 0.07112790198184574


### Idea of solution

I ran a for loop through each key of p, knowing that the keys of p and q are the same. For each key of p (and q), I executed the computation p_i(log(p_i/q_i)) if p_i is not 0 since log(0) does not exist. I added the result to a vairable that I intialized to 0 to get the total sum of all terms. This variable was returned.

### Discussion

Looking at the tests, we can see that not all of the distributions are non-negative which either means that I have an error in the calculations which I could not locate or that the probabilities are too small to get an accurate sum of logs.
It appears that knowing the predicted and uniform distributions helps in encoding the original distribution greatly but the original distribution does not help the other two. The predicted and uniform distributions don't seem to help encode eachother since both their divergences are near zero.

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [86]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

-0.254, +0.451, +0.267, -0.493
+0.243, +0.496, +0.084, -0.143


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [87]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[-0.00537851  0.11101195 -0.49957744 -0.30492211]
 [ 0.01817657  0.39433265 -0.49701862 -0.0581689 ]]
Using [0.02, 0.39, -0.50, -0.06] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.33062863
KL divergence of stationary distribution prefix of length    10 is 0.38992107
KL divergence of stationary distribution prefix of length   100 is 0.61068418
KL divergence of stationary distribution prefix of length  1000 is 0.98421894
KL divergence of stationary distribution prefix of length 10000 is 0.59148237


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [88]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[-0.12974871  0.250996    0.35271711 -0.38187202]

Using [-0.11956792 -0.03414162 -0.19941102 -0.24758042] as initial distribution:
kl-divergence   empirical distribution
0.37637114852   [ 0.28153457 -0.29691819  0.48676966  0.05713638]
0.56404697319   [ 0.0088926  -0.15564204  0.03424995  0.00070839]
0.45667336726   [ 0.29587369 -0.39647587  0.37758201 -0.4903873 ]
0.35806329526   [0.36256878 0.40083195 0.18905145 0.20282102]
0.03121525022   [ 0.29721744  0.14385157 -0.02082882 -0.17573513]

Using [0.33204736 0.43776623 0.41229545 0.15427126] as initial distribution:
kl-divergence   empirical distribution
0.83861464076   [ 0.04612454 -0.45099859  0.39856767 -0.31792474]
0.29395666369   [ 0.07573083 -0.19558419 -0.37272953  0.39743735]
0.73765720222   [ 0.36292045 -0.25453473  0.48171657  0.00637594]
0.40992860587   [-0.17814365  0.2600352

### Idea of solution

fill in

### Discussion
fill in